In [1]:
import sys
from pathlib import Path
import torch
import requests
import json
import os 

model_name="large-v2"
speaker_id="M02"
test_speaker=speaker_id

In [2]:
from dataclasses import dataclass, field, replace
from typing import TYPE_CHECKING, Dict, Iterable, List, Optional, Sequence, Tuple, Union
import whisper_openAI.whisper as whisper
import torch
from whisper_openAI.whisper.tokenizer import Tokenizer, get_tokenizer
import torch
import torch.nn.functional as F
from torch import Tensor

# We get the acoustic embeddings from Whisper Large V2
model,processor = whisper.load_model("large-v2")
# model,processor = whisper.load_model("medium")

In [3]:
model.eval()

Whisper(
  (encoder): AudioEncoder(
    (conv1): Conv1d(80, 1280, kernel_size=(3,), stride=(1,), padding=(1,))
    (conv2): Conv1d(1280, 1280, kernel_size=(3,), stride=(2,), padding=(1,))
    (blocks): ModuleList(
      (0-31): 32 x ResidualAttentionBlock(
        (attn): MultiHeadAttention(
          (query): Linear(in_features=1280, out_features=1280, bias=True)
          (key): Linear(in_features=1280, out_features=1280, bias=False)
          (value): Linear(in_features=1280, out_features=1280, bias=True)
          (out): Linear(in_features=1280, out_features=1280, bias=True)
        )
        (attn_ln): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
        (mlp): Sequential(
          (0): Linear(in_features=1280, out_features=5120, bias=True)
          (1): GELU(approximate='none')
          (2): Linear(in_features=5120, out_features=1280, bias=True)
        )
        (mlp_ln): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
      )
    )
    (ln_post): LayerNorm((

In [4]:
import json

# The below is the json file you can generate using the "To generatn-best hyporhesis.ipynb" notebook; Need to further tokenize the hypothesis

with open(f'Inference/gs_inferences/torgo_train_{speaker_id}_{model_name}.json', "r") as file:  # Change the file path and name here
    train_data = json.load(file)

with open(f'Inference/gs_inferences/torgo_val_{speaker_id}_{model_name}.json', "r") as valid_file:
    val_data = json.load(valid_file)

# Load the test set
with open(f'Inference/gs_inferences/torgo_test_{speaker_id}_{model_name}.json', "r") as test_file:
    test_data = json.load(test_file)

In [5]:
train_data[15]

{'F01-Session1-arrayMic-0034': {'temp': 0.9299999999999999,
  'path': '/work/van-speech-nlp/data/torgo/F01/Session1/wav_arrayMic/0034.wav',
  'ground_truth': 'feet',
  'inference': ['be',
   'dean',
   'like that girl from tenai yeah please',
   'ping',
   'please',
   'please believe me',
   'raf',
   'see',
   'well wipe you clean'],
  'source': 'NP-Torgo',
  'category': 'NP-Torgo',
  'time': 30.0}}

In [6]:


from lit_llama.tokenizer import Tokenizer
from tqdm import tqdm


tokenizer_path: Path = Path("weights/tokenizer.model")
tokenizer = Tokenizer(tokenizer_path)
print(f"train has {len(train_data):,} samples")


/work/van-speech-nlp/jindaznb/mmenv/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/work/van-speech-nlp/jindaznb/mmenv/lib/python3.9/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
/work/van-speech-nlp/jindaznb/jslpnb/mllm_expriments/slam-llm/transformers/src/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


train has 9,402 samples


In [7]:
import torch
old_data_train = torch.load(f'Inference/gs_inferences/baseline_data_tiny_hypo_v2/torgo_{test_speaker}_train.pt',map_location=torch.device('cpu'))
old_data_val = torch.load(f'Inference/gs_inferences/baseline_data_tiny_hypo_v2/torgo_{test_speaker}_val.pt',map_location=torch.device('cpu'))
old_data_test = torch.load(f'Inference/gs_inferences/baseline_data_tiny_hypo_v2/torgo_{test_speaker}_test.pt',map_location=torch.device('cpu'))

# check if data index are same

In [8]:
old_data_train[15]

{'temp': 0.77,
 'path': '/work/van-speech-nlp/data/torgo/F01/Session1/wav_arrayMic/0034.wav',
 'ground_truth': 'feet',
 'inference': ['ah ah',
  'b',
  'beee',
  'beep',
  'beep!',
  'boom',
  'bye',
  'c',
  'cf',
  'cuuu',
  'everything',
  'f',
  'fear',
  'fee',
  'fee!',
  'fee!!',
  'feeeee',
  'feeeeeee!',
  'feeeeeeee',
  'feeeing',
  'feez!',
  'fehm',
  'female',
  'ferien',
  'fie',
  'fii',
  'fii!',
  'fiii',
  'fiiiie',
  'fiiit!',
  'fiim',
  'fjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjj',
  'hehe!',
  'here',
  'hey',
  'hey!',
  'hmm',
  'i must know that its just like a clown',
  'i think',
  'it p l',
  'its',
  'its been a while',
  'lil lil lil lil lil lil lil lil lil lil lil lil lil',
  'lucky!',
  'me',
  'ok!',
  'okay…',
  'p',
  'pee',
  'peeeeee',
  'peein!!',
  'phi',
  'please',
 

In [9]:
# Print the lengths of the loaded data
print(f'Length of old_data_train: {len(old_data_train)}')
print(f'Length of old_data_val: {len(old_data_val)}')
print(f'Length of old_data_test: {len(old_data_test)}')

Length of old_data_train: 9402
Length of old_data_val: 460
Length of old_data_test: 403


In [10]:
stop

NameError: name 'stop' is not defined

In [ ]:
def tokenize(tokenizer: Tokenizer, string: str, max_length: int, eos=True) -> torch.Tensor:
    return tokenizer.encode(string, bos=True, eos=eos, max_length=max_length)
    
def process_train_data(train_data, old_data):
    instruction = 'You are an ASR transcript selector. You have a few transcripts generated by an automatic speech recognition model. Your task is to generate the most likely transcript from them. If the generated transcripts have grammatical or logical errors, you will modify them accordingly to produce the most accurate and coherent transcript.'
    result = []

    for i in tqdm(range(len(train_data))):        
        for name in train_data[i].keys():
            ip = train_data[i][name]
        inference = ip['inference']
        gt = ip['ground_truth']
            
        # Removing the ground_truth, if present among the inferences for the prompt
        if gt in inference:
            inference.remove(gt)
                
        # Joining the inputs with '\n'
        for_input = '\n'.join(inference[:15])
        # The prompt follows the Alpaca template
        full_prompt = f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\n{instruction}\n\n### Input:\n{for_input}\n\n### Response:"""
        full_prompt_and_response = full_prompt + gt

        encoded_full_prompt = tokenize(tokenizer, full_prompt, max_length=2048, eos=False)
        encoded_full_prompt_and_response = tokenize(tokenizer, full_prompt_and_response, eos=True, max_length=2048)
        labels = encoded_full_prompt_and_response.clone()
        labels_with_masked_input = encoded_full_prompt_and_response.clone()
        labels_with_masked_input[:len(encoded_full_prompt)] = -1
        
        # path = ip['path']
        # audio = whisper.load_audio(path)  
        # audio = whisper.pad_or_trim(audio)            
        # mel = whisper.log_mel_spectrogram(audio).to(model.device)  # Adjust as needed for your model
        # mel = mel.unsqueeze(0)
        
        # with torch.no_grad():
        #     audio_features = model.encoder(mel)
        audio_features = old_data[i]["audio_features"]
        
        result.append({**ip, 'index': name, "input_ids": encoded_full_prompt_and_response, "input_ids_no_response": encoded_full_prompt, "labels": labels, 'labels_with_masked_input': labels_with_masked_input, 'audio_features': audio_features.bfloat16()})
    print(len(result))
    return result

In [ ]:
split = "train"
result = process_train_data(train_data, old_data_train)
torch.save(result,f'Inference/gs_inferences/torgo_{speaker_id}_{model_name}_{split}.pt')
print(f"Processed {split} data and saved checkpoint for {speaker_id}")

split = "val"
result = process_train_data(val_data, old_data_val)
torch.save(result,f'Inference/gs_inferences/torgo_{speaker_id}_{model_name}_{split}.pt')
print(f"Processed {split} data and saved checkpoint for {speaker_id}")

split = "test"
result = process_train_data(test_data, old_data_test)
torch.save(result,f'Inference/gs_inferences/torgo_{speaker_id}_{model_name}_{split}.pt')
print(f"Processed {split} data and saved checkpoint for {speaker_id}")